## requests 

In [ ]:
from bs4 import BeautifulSoup
import requests
url = "https://omni.se/fangelse-pa-obestamd-tid-kan-domas-ut-fran-i-var/a/BxloXl"

response = requests.get(url)

response

<Response [200]>

<h1 class="Title_articleTitle__EYHH0">Fängelse på obestämd tid kan dömas ut från i vår</h1><div class="Byline_bylineContainer__WhCT8">Av<!-- --> <span><span>Kinga Sandén</span></span><br/>Publicerad <time class="Timestamp_timestamp__9fbnE" datetime="2026-01-22T14:21:53Z" role="note">22 januari, 15:21</time></div><div class="Ad_adContainer__suvpO"><button class="AdInfoWrapper_label__TkUn9 AdInfoWrapper_marking__TbxnW"><span>Annons</span><span class="AdInfoWrapper_icon__LAloN"><i class="Icon_icon__ZG9ss Icon_normal__i1rrW Icon_iconColor__T17Qe"><svg aria-label="Inställningar" data-icon-variant="dots" fill="none" focusable="false" height="100%" role="img" viewbox="0 0 24 24" width="100%" xmlns="http://www.w3.org/2000/svg"><path d="M17 12h.01M12 12h.01M7 12h.01M22 12c0 5.523-4.477 10-10 10S2 17.523 2 12 6.477 2 12 2s10 4.477 10 10" stroke="#202020" stroke-linecap="round" stroke-linejoin="round" stroke-width="1.5"></path></svg></i></span></button><div class="Ad_placement__QjKHs Ad_articleResource__4MgjU" data-testid="ad-component"><div class="Ad_wrapper__tKQZh Ad_articleWrapper__vrhsd"><div class="Ad_loader__OfrTR" id="" style="height:0;width:0"></div></div></div></div><div class="Text_textContainer__d___Z Text_news___b1pP"><p>Ett nytt fängelsestraff införs som innebär inlåsning på obestämd tid. Det meddelar regeringen på en pressträff, enligt TT. </p><p>– Vissa brottslingar är så farliga att de inte bör släppas ut, säger justitieminister Gunnar Strömmer i en kommentar, och nämner serievåldtäktsmän som exempel. </p><p>Särskilt farliga brottslingar med hög återfallsrisk ska kunna dömas till ”säkerhetsförvaring” från och med den 15 april. Det innebär att domstolen bestämmer en minimitid i fängelse och därtill en ramtid på ytterligare fyra till sex år. Ramtiden kan sedan förlängas med tre år i taget för att ”skydda samhället”. Detta oavsett om livstid ingår i straffskalan för deras brott. </p></div>

## Strukturera med BeautifulSoup
- Jättejobbigt, så extremt mycket text följer med, speciellt för annonser. 

In [ ]:
url = "https://omni.se/"

full_response = requests.get(url)

soup = BeautifulSoup(full_response.text, "html.parser")

# soup

In [ ]:

titles = soup.find_all("h2") 
titles

[<h2 class="TeaserHeading_teaserTitle__ShOGe TeaserHeading_large__bsoW0">Iran förnekar Trumputspel: ”Inga stoppade avrättningar”</h2>,
 <h2 class="TeaserHeading_teaserTitle__ShOGe">Dödssiffran stiger – 5 000 uppges ha dödats i Iran</h2>,
 <h2 class="TeaserHeading_teaserTitle__ShOGe TeaserHeading_large__bsoW0">80-åring hittad i källare i Helsingfors – har levt i fönsterlöst rum i decennier</h2>,
 <h2 class="TeaserHeading_teaserTitle__ShOGe TeaserHeading_large__bsoW0">Så påverkas du av nya skatteändringen</h2>,
 <h2 class="TeaserHeading_teaserTitle__ShOGe TeaserHeading_large__bsoW0">Krigets första trepartsmöte inlett – Donbas nyckelfråga</h2>,
 <h2 class="TeaserHeading_teaserTitle__ShOGe">USA, Ryssland och Ukraina möts i samtal i Abu Dhabi</h2>,
 <h2 class="TeaserHeading_teaserTitle__ShOGe">Zelenskyj: Avtal med garantier klart – men inte påskrivet</h2>,
 <h2 class="TeaserHeading_teaserTitle__ShOGe"><span class="TeaserHeading_titleVignette__qr53g TeaserHeading_premium__zebOj">Omni förklar

## Scraping med firecrawl hos omni.se/inrikes. Kostar 1 credit per sida
- Tar hela sidan med all synlig text, tar inte mer artiklar som dyker om när man scrollat längst ner. 

In [ ]:
from firecrawl import FirecrawlApp
from dotenv import load_dotenv
import os
load_dotenv()
app = FirecrawlApp(api_key=os.getenv("CRAWL_KEY"))

url = "https://omni.se/inrikes"
scrape_result = app.scrape(url, formats= ["markdown", "html"], only_main_content=True) #ska skippa länkar och menyer

print(scrape_result)

print()

In [ ]:
with open("scrape_1.txt", "w", encoding="utf8") as file: 
    file.write(scrape_result.markdown)

## Crawling - Exctractar all content samt alla tillängliga sub-pages. 
 

In [ ]:
from firecrawl import Firecrawl
url = "https://omni.se/inrikes"

app = Firecrawl(api_key=os.getenv("CRAWL_KEY"))
docs = app.crawl(url=url)
print(docs)

In [36]:
with open("scrape_2.txt", "w", encoding="utf8") as file: 
    for d in docs.data: 
        file.write(d.markdown) 

In [ ]:
from firecrawl import Firecrawl
url = "https://omni.se/inrikes"

app = Firecrawl(api_key=os.getenv("CRAWL_KEY"))

full_crawl = app.crawl(url, limit=10, scrape_options={"formats": ["markdown"]})

full_crawl

In [ ]:
for doc in full_crawl.data:
    print(doc.metadata.title)
    print(full_crawl.data[0].markdown)

## Nu lyckades den! 
- 0 är bara förstasidan med ingresser
- 1-9 är artiklar

In [ ]:
from firecrawl import Firecrawl
url = "https://omni.se/inrikes"

app = Firecrawl(api_key=os.getenv("CRAWL_KEY"))

full_crawl = app.crawl(url, 
                    limit=10, 
                    # include_paths=["inrikes","a/*"], #tillåter länkar till artiklar
                    crawl_entire_domain=True,
                    max_discovery_depth=1, #ta bara en länk in från startsidan
                    scrape_options={"formats": ["markdown"]},
                    
                    )

# full_crawl


## Spara till fil

In [ ]:
import json
type(full_crawl)

json_data = full_crawl.model_dump_json(indent=4)

with open("full_crawl_9.json", "w", encoding="utf8") as file:
    file.write(json_data)

#### doc.metadata är ett Pydantic-objekt. Du kan inte hämta saker med .get, utan bara via punktannotation.
#### Saker du kan hämta ut från metadata: 

In [110]:
full_crawl.data[1].metadata.model_dump().keys()


dict_keys(['title', 'description', 'url', 'language', 'keywords', 'robots', 'og_title', 'og_description', 'og_url', 'og_image', 'og_audio', 'og_determiner', 'og_locale', 'og_locale_alternate', 'og_site_name', 'og_video', 'favicon', 'dc_terms_created', 'dc_date_created', 'dc_date', 'dc_terms_type', 'dc_type', 'dc_terms_audience', 'dc_terms_subject', 'dc_subject', 'dc_description', 'dc_terms_keywords', 'modified_time', 'published_time', 'article_tag', 'article_section', 'source_url', 'status_code', 'scrape_id', 'num_pages', 'content_type', 'proxy_used', 'timezone', 'cache_state', 'cached_at', 'credits_used', 'concurrency_limited', 'concurrency_queue_duration_ms', 'error', 'twitter:card', 'twitter:creator', 'og:site_name', 'og:image:width', 'application-name', 'og:title', 'og:locale', 'twitter:title', 'viewport', 'twitter:image:height', 'apple-itunes-app', 'article:modified_time', 'og:image', 'og:image:height', 'og:url', 'twitter:site', 'og:description', 'twitter:image:alt', 'og:type', 'a

### Saker i metadata som innehåller något

In [108]:
for key, value in full_crawl.data[1].metadata.model_dump().items(): 
    if value is not None: 
        print(key, ": ", value)

title :  ”Stor delseger” för Trädgården – blir kvar i två år
description :  Allting pekar på att nattklubben Trädgården i Stockholm räddas, rapporterar P4 Stockholm. Stockholms stad har genomfört flera åtgärder för att klubben ska kunna fortsätta sin verksamhet, oavsett vad mark- och miljödomstolen kommer fram till under hösten.
url :  https://omni.se/stor-delseger-for-tradgarden-blir-kvar-i-tva-ar/a/KMoe4y
language :  sv
og_title :  ”Stor delseger” för Trädgården – blir kvar i två år
og_description :  Allting pekar på att nattklubben Trädgården i Stockholm räddas, rapporterar P4 Stockholm. Stockholms stad har genomfört flera åtgärder för att klubben ska kunna fortsätta sin verksamhet, oavsett vad mark- och miljödomstolen kommer fram till under hösten.
og_url :  https://omni.se/stor-delseger-for-tradgarden-blir-kvar-i-tva-ar/a/KMoe4y
og_image :  https://gfx.omni.se/images/6e710d00-02b0-48ed-94c0-05e75b23d1ce?h=708&tight=false&w=1372
og_locale :  sv_SE
og_site_name :  Omni
favicon :  ht

In [ ]:
metadata_dict = full_crawl.data[1].metadata.model_dump()
metadata_dict.get("article:section") ## Hämta ut saker som har keys name:name

'Nöje & kultur'

## Exempel på roliga saker man kan få ut från crawlen, mer än artikelinnehåll

In [153]:

i = 0
for doc in full_crawl.data: 
    # print(doc.markdown)
    metadata_dict = doc.metadata.model_dump()
    og_type = metadata_dict.get("og:type")
    if og_type == "article":
        
        section = metadata_dict.get("article:section")
        article_image_link = metadata_dict.get("og:image")
        article_image_description = metadata_dict.get("og:image:alt")
        teaser_text = metadata_dict.get("description")

        print(i,"-" * 60, )
        print("title: ", doc.metadata.title)
        print("published time: ", doc.metadata.published_time) 
        print("section: ", section)
        print("image link: ", article_image_link)
        print("image description: ", article_image_description)
        print("type: ", og_type)
        print("teaser text: ", teaser_text)
        i +=1
    else: 
        print(i, " Var inte en artikel")
        i+=1
    
# print(full_crawl.metadata.title)
# print(full_crawl.data[1])

0  Var inte en artikel
1 ------------------------------------------------------------
title:  ”Stor delseger” för Trädgården – blir kvar i två år
published time:  2025-09-11T10:15:06Z
section:  Nöje & kultur
image link:  https://gfx.omni.se/images/6e710d00-02b0-48ed-94c0-05e75b23d1ce?h=708&tight=false&w=1372
image description:  Arkivbild på klubben Trädgården vid Skanstull i Stockholm.
type:  article
teaser text:  Allting pekar på att nattklubben Trädgården i Stockholm räddas, rapporterar P4 Stockholm. Stockholms stad har genomfört flera åtgärder för att klubben ska kunna fortsätta sin verksamhet, oavsett vad mark- och miljödomstolen kommer fram till under hösten.
2  Var inte en artikel
3 ------------------------------------------------------------
title:  Snöilska mot Skistar: ”Det går inte att åka”
published time:  2025-04-18T14:17:02Z
section:  Inrikes
image link:  https://gfx.omni.se/images/0fe95a7c-53e4-4780-8a36-a084abd18f11?h=708&tight=false&w=1372
image description:  Bild från 

## Vad vill man ha med? 

In [ ]:
url_svt = "https://www.svt.se/nyheter/arkiv/2025/11/20/"

In [ ]:
# Sök efter SVT-artiklar från november
search_result = app.search(
    query="site:svt.se/nyheter",
    limit=3,
    tbs="cdr:1,cd_min:11/20/2025,cd_max:11/21/2025"
)
# search_result # Länkar för artiklar på alla dessa datum

In [211]:
type(search_result)
for item in search_result.web: 
    print(item)

url='https://www.svt.se/nyheter/utrikes/direktrapport-forsamrat-sakerhetslage?inlagg=f9a0fa5a4b5f4d2a99a73bca0305f47c' title='Senaste nytt om kriget i Ukraina' description='Rysslands storskaliga invasion av Ukraina har pågått sedan den 24 februari 2022, då ryska trupper anföll Ukraina från flera håll.' category=None
url='https://www.svt.se/nyheter/inrikes/svt-vinner-stora-journalistpriset-2025' title='SVT vinner Stora journalistpriset 2025' description='SVT kammar hem Stora journalistpriset i kategorin ”Årets Förnyare” för dokumentären Gaza – till sista andetaget. – Vi är förstås otroligt glada för att ...' category=None
url='https://www.svt.se/nyheter/utrikes/direktrapport-forsamrat-sakerhetslage?inlagg=5f4ebbd989cea04b08ad5f643bcc6a1e' title='Senaste nytt om kriget i Ukraina | SVT ...' description='Trump: Zelenskyj måste acceptera avtalet. USA:s president Donald Trump fick frågor om fredsplanen på 28 punkter under mötet med New Yorks tillträdande ...' category=None


In [ ]:
for item in search_result.web: 
    crawl_result = app.crawl(item.url, 
                    limit=10, 
                    # include_paths=["inrikes","a/*"], #tillåter länkar till artiklar
                    crawl_entire_domain=True,
                    max_discovery_depth=1, #ta bara en länk in från startsidan
                    scrape_options={"formats": ["markdown"]},
                    
                    )

In [ ]:
crawl_result[1]

In [227]:
search_result.web[1].description

'SVT kammar hem Stora journalistpriset i kategorin ”Årets Förnyare” för dokumentären Gaza – till sista andetaget. – Vi är förstås otroligt glada för att ...'

In [228]:
# Skrapa datumen
i = 0
scrape_list = []
for item in search_result.web: 

    print(i, " ", item.title)
    i +=1
    scrape_result = app.scrape(
        url=item.url,
            formats= ['markdown', 'html'],  # Välj format (markdown, html, rawHtml)
            only_main_content = True    # Tar bort menyer och annonser automatiskt
    )
    scrape_list.append(scrape_result.markdown)
    
# scrape_result[0]

# Kom åt datan
# print(scrape_result.markdown)
# print(scrape_result.metadata.title)

0   Senaste nytt om kriget i Ukraina
1   SVT vinner Stora journalistpriset 2025
2   Senaste nytt om kriget i Ukraina | SVT ...


### Crawla omni igen, fast med limit 100
- får ut ca. 26-29 artiklar mellan de relevanta datumen 20/11-2025 till idag, på limit=100. 

In [ ]:
url = "https://omni.se/tech"
full_crawl = app.crawl(url, 
                    limit=100, 
                    crawl_entire_domain=True,
                    max_discovery_depth=1, #ta bara en länk in från startsidan
                    scrape_options={"formats": ["markdown"]},
                    )

In [ ]:
nice_crawls = []
for i, item in enumerate(full_crawl.data): 
    metadata_dict = item.metadata.model_dump()
    og_type = metadata_dict.get("og:type")
    if str(item.metadata.published_time) > "2025-11-19" and item.metadata.published_time is not None:
        print(item.metadata.published_time, og_type)
        nice_crawls.append(item)
        
print(len(nice_crawls))
json_data =[item.model_dump() for item in nice_crawls]
with open("tech_100.json", "w", encoding="utf8") as file: 
        json.dump(json_data, file, indent=4)

2026-01-27T07:36:52Z article
2026-01-08T16:27:53Z article
2026-01-27T11:13:11Z article
2026-01-25T15:27:50Z article
2026-01-27T07:35:19Z article
2026-01-09T18:27:16Z article
2026-01-27T11:41:35Z article
2026-01-10T15:46:28Z article
2026-01-26T15:48:50Z article
2026-01-23T18:30:38Z article
2026-01-27T11:02:49Z article
2026-01-26T22:08:28Z article
2026-01-24T22:00:48Z article
2026-01-27T11:58:31Z article
2026-01-27T08:52:45Z article
2025-12-29T08:06:42Z article
2025-12-04T06:07:24Z article
2026-01-27T11:28:36Z article
2026-01-27T12:31:53Z article
2026-01-27T09:14:11Z article
2026-01-27T05:47:21Z article
2026-01-27T07:59:21Z article
2026-01-27T09:51:00Z article
2026-01-13T17:27:02Z article
2026-01-26T19:39:28Z article
2026-01-25T19:54:26Z article
2026-01-15T19:29:03Z article
2026-01-21T16:06:42Z article
2026-01-24T15:53:59Z article
29
